<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# Resample Operators with Pangeo

<img src="https://raw.githubusercontent.com/pangeo-data/pangeo.io/refs/heads/main/public/Pangeo-assets/pangeo_logo.png"
     alt="Pangeo logo"
     style="float: center; margin-right: 9px;" />

## Resample Operators

Sometimes we need to align the spatial or temporal dimension of two datacubes, so that they can be merged together.

### Spatial resampling Sentinel-2 to match Landsat

Start importing the necessary libraries

In [1]:
# STAC Catalogue Libraries
import pystac_client
import stackstac
from rasterio.enums import Resampling

Create two datacubes, one for Sentinel-2 and one for Landsat

In [2]:
spatial_extent = [11.4, 45.5, 11.42, 45.52]
temporal_extent = ["2023-06-01", "2023-06-30"]

## Datacube for Sentinel-2

In [3]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
s2_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["sentinel-2-l2a"]
).item_collection()

s2_cube = stackstac.stack(s2_items,
                     bounds_latlon=spatial_extent,
                     assets=["red","nir"]
)
s2_cube

/home/conda/cubes-and-clouds/30d8bce6e9d2d3312ad568684b7851b268e5321683bef302b527561c66bcbe23-20240905-143316-989438-487-pangeo/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-dabb139897fe2b1c1b599452370e1c47' (time: 6,
                                                                band: 2,
                                                                y: 228, x: 164)> Size: 4MB
dask.array<fetch_raster_window, shape=(6, 2, 228, 164), dtype=float64, chunksize=(1, 1, 228, 164), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 48B 2023-0...
    id                                       (time) <U24 576B 'S2A_32TPR_2023...
  * band                                     (band) <U3 24B 'red' 'nir'
  * x                                        (x) float64 1kB 6.874e+05 ... 6....
  * y                                        (y) float64 2kB 5.044e+06 ... 5....
    s2:generation_time                       (time) <U27 648B '2023-06-02T16:...
    ...                                       ...
    gsd                                      int64 8B 10
    proj:transform                           object 8B {0, 5100000, 600000, 1...
    common_name                              (band) <U3 24B 'red' 'nir'
    center_wavelength                        (band) float64 16B 0.665 0.842
    full_width_half_max                      (band) float64 16B 0.038 0.145
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(687430.0, 5041290.0, 689070.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 687430.00|\n| 0.00,-10.00, 5043570.00|\n| 0.0...
    resolution:  10.0

## Datacube for Landsat

In [4]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1"
catalog = pystac_client.Client.open(URL)
l8_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["landsat-c2-l2"]
).item_collection()

l8_cube = stackstac.stack(l8_items,
                     bounds_latlon=spatial_extent,
                     assets=["red","nir08"]
)
l8_cube

/home/conda/cubes-and-clouds/30d8bce6e9d2d3312ad568684b7851b268e5321683bef302b527561c66bcbe23-20240905-143316-989438-487-pangeo/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-6cac7d80861cd439c8442a313b8dd112' (time: 15,
                                                                band: 2, y: 76,
                                                                x: 55)> Size: 1MB
dask.array<fetch_raster_window, shape=(15, 2, 76, 55), dtype=float64, chunksize=(1, 1, 76, 55), chunktype=numpy.ndarray>
Coordinates: (12/36)
  * time                         (time) datetime64[ns] 120B 2023-06-03T10:03:...
    id                           (time) <U31 2kB 'LC08_L2SP_193028_20230603_0...
  * band                         (band) <U5 40B 'red' 'nir08'
  * x                            (x) float64 440B 6.874e+05 ... 6.89e+05
  * y                            (y) float64 608B 5.044e+06 ... 5.041e+06
    landsat:correction           <U4 16B 'L2SP'
    ...                           ...
    title                        (band) <U22 176B 'Red Band' 'Near Infrared B...
    raster:bands                 object 8B {'nodata': 0, 'data_type': 'uint16...
    storage:region               <U9 36B 'us-west-2'
    storage:platform             <U3 12B 'AWS'
    storage:requester_pays       bool 1B True
    epsg                         int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(687420.0, 5041290.0, 689070.0...
    crs:         epsg:32632
    transform:   | 30.00, 0.00, 687420.00|\n| 0.00,-30.00, 5043570.00|\n| 0.0...
    resolution:  30.0

From the previous outputs, notice the shape difference in the spatial dimensions `x` and `y`.

This is due to the different resolution of the two collections: 10m for Sentinel-2, 30m for Landsat.

Now we use the `stackstac.stack` to resample the Sentinel-2 data to match Landsat.

In [5]:
s2_cube_coarse = stackstac.stack(s2_items,
                                 bounds_latlon=spatial_extent,
                                 assets=["red","nir"],
                                 resolution=30.0, resampling=Resampling.average, epsg=32632)
s2_cube_coarse

/home/conda/cubes-and-clouds/30d8bce6e9d2d3312ad568684b7851b268e5321683bef302b527561c66bcbe23-20240905-143316-989438-487-pangeo/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-662fb395897849774985f5bd27a315e1' (time: 6,
                                                                band: 2, y: 76,
                                                                x: 55)> Size: 401kB
dask.array<fetch_raster_window, shape=(6, 2, 76, 55), dtype=float64, chunksize=(1, 1, 76, 55), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 48B 2023-0...
    id                                       (time) <U24 576B 'S2A_32TPR_2023...
  * band                                     (band) <U3 24B 'red' 'nir'
  * x                                        (x) float64 440B 6.874e+05 ... 6...
  * y                                        (y) float64 608B 5.044e+06 ... 5...
    s2:generation_time                       (time) <U27 648B '2023-06-02T16:...
    ...                                       ...
    gsd                                      int64 8B 10
    proj:transform                           object 8B {0, 5100000, 600000, 1...
    common_name                              (band) <U3 24B 'red' 'nir'
    center_wavelength                        (band) float64 16B 0.665 0.842
    full_width_half_max                      (band) float64 16B 0.038 0.145
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(687420.0, 5041290.0, 689070.0...
    crs:         epsg:32632
    transform:   | 30.00, 0.00, 687420.00|\n| 0.00,-30.00, 5043570.00|\n| 0.0...
    resolution:  30.0